### Notebook for concept detection in neural network

In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.concept_tsumego

CONCEPT_NAME = concepts.concept_tsumego.__name__


2023-10-26 13:11:40.185725: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [13]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0
    random_moves = False

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)

        if random_moves:
            action = go_env.uniform_random_action()
        else:
            action = agents[current_player].best_action(state)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

CASES_TO_SAMPLE = 1000 # 25000

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE or len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)
            positive_cases.extend(pos)
            negative_cases.extend(neg)
            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases:   1%|          | 6/1000 [00:07<20:00,  1.21s/it]
Positive cases: 1002it [21:24,  1.13it/s]                         

In [14]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (1000, 4, 5, 5)
Negative cases:  (1000, 4, 5, 5)


In [15]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [16]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  964
Duplicate count in negative:  308


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [17]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 1600


In [18]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = concepts.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Epoch 1/50
50/50 [==============================] - 0s 3ms/step - loss: 0.8886 - val_loss: 0.8349
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8734 - val_loss: 0.8258
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8599 - val_loss: 0.8182
Epoch 4/50
50/50 [==============================] - 0s 3ms/step - loss: 0.8485 - val_loss: 0.8110
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8381 - val_loss: 0.8045
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8288 - val_loss: 0.7984
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8203 - val_loss: 0.7928
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8126 - val_loss: 0.7874
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8054 - val_loss: 0.7821
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7988 - val_loss: 0.7769
Epoch 11/50
50/50 [

In [19]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050]

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_resblock_outputs(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Getting resblock outputs: 100%|██████████| 63/63 [00:00<00:00, 249.33it/s]


Performing regression for layer 0
Epoch 1/50
50/50 [==============================] - 0s 3ms/step - loss: 1.0007 - val_loss: 0.9930
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9754 - val_loss: 0.9676
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9517 - val_loss: 0.9435
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9291 - val_loss: 0.9211
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9078 - val_loss: 0.8995
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8876 - val_loss: 0.8795
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8684 - val_loss: 0.8607
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8506 - val_loss: 0.8431
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8337 - val_loss: 0.8265
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8179 - va

Getting resblock outputs: 100%|██████████| 63/63 [00:00<00:00, 283.85it/s]

Performing regression for layer 0
Epoch 1/50


50/50 [==============================] - 0s 3ms/step - loss: 0.9886 - val_loss: 0.9162
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9305 - val_loss: 0.8962
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9042 - val_loss: 0.8808
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8833 - val_loss: 0.8653
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8645 - val_loss: 0.8496
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8468 - val_loss: 0.8342
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8303 - val_loss: 0.8201
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8151 - val_loss: 0.8068
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8000 - val_loss: 0.7927
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7862 - val_loss: 0.7800
Epoch 11/50
50/50 [===========

...

Performing regression for layer 0
Epoch 1/50


50/50 [==============================] - 0s 3ms/step - loss: 0.9682 - val_loss: 0.9213
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9063 - val_loss: 0.8820
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8663 - val_loss: 0.8469
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8323 - val_loss: 0.8143
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8023 - val_loss: 0.7867
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7759 - val_loss: 0.7617
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7516 - val_loss: 0.7387
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7297 - val_loss: 0.7165
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7100 - val_loss: 0.6985
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6918 - val_loss: 0.6813
Epoch 11/50
50/50 [===========

Getting resblock outputs: 100%|██████████| 63/63 [00:00<00:00, 279.80it/s]

Performing regression for layer 0
Epoch 1/50


50/50 [==============================] - 0s 3ms/step - loss: 0.9398 - val_loss: 0.9151
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8796 - val_loss: 0.8764
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8382 - val_loss: 0.8460
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8043 - val_loss: 0.8201
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7745 - val_loss: 0.7963
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7487 - val_loss: 0.7751
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7256 - val_loss: 0.7560
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7047 - val_loss: 0.7387
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6854 - val_loss: 0.7210
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6678 - val_loss: 0.7065
Epoch 11/50
50/50 [===========